In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
import pickle

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [2]:
data = pd.read_csv('D:\\庫存健診開發\\data\\processed\\TWSE_feature_all_5day.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,VWAP_day5_return_5
0,2007-08-21,1101,台泥,37.50,39.90,36.80,37.5,-0.79,8.20,42969.0,...,4118.371040,1138.194426,-106.652476,176.739261,0.486939,0.459669,0.356176,0.310119,0.450905,-9.31
1,2007-08-22,1101,台泥,37.35,38.40,36.80,38.2,1.87,4.27,23201.0,...,4014.678434,1136.459691,-104.736206,-245.989525,0.486939,0.591611,0.473894,0.277664,0.393158,-9.45
2,2007-08-23,1101,台泥,39.60,40.00,39.25,40.0,4.71,1.96,30456.0,...,4403.772485,1231.769792,-86.190952,1427.473280,0.558479,0.953949,0.713921,0.328728,0.371681,-6.01
3,2007-08-24,1101,台泥,40.00,40.85,39.65,39.8,-0.50,3.00,32479.0,...,3990.608891,1374.195281,-90.830337,2613.957456,0.592940,0.831259,0.772590,0.438425,0.393929,-0.82
4,2007-08-27,1101,台泥,41.00,41.00,39.20,39.5,-0.75,4.52,16646.0,...,3705.511740,1349.949967,-160.328628,4073.934540,0.566202,0.719624,0.746107,0.470497,0.419452,1.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318889,2019-09-02,9958,世紀鋼,69.40,71.60,68.70,71.6,3.32,4.18,1980.0,...,10.399445,30.535589,5.001610,-135.261368,0.433530,0.618687,0.459431,0.251528,0.272107,-2.98
2318890,2019-09-03,9958,世紀鋼,71.70,75.80,71.30,74.1,3.49,6.28,7286.0,...,7.350933,30.535589,35.547803,270.566486,0.669457,0.824588,0.642009,0.472498,0.338904,0.48
2318891,2019-09-04,9958,世紀鋼,75.00,75.40,74.40,74.9,1.08,1.35,4513.0,...,2.819805,30.535589,35.061530,348.691073,0.669457,0.884340,0.763175,0.644186,0.440665,3.30
2318892,2019-09-05,9958,世紀鋼,75.50,75.70,74.00,74.2,-0.93,2.27,2186.0,...,14.334574,30.535589,34.972803,301.521091,0.636482,0.815226,0.789200,0.739897,0.540409,5.25


In [3]:
df = data[(data.ts.dt.date >= date(2017,9,1)) & (data.ts.dt.date <= date(2019,9,6))]
df

,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,foreign_pvt,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,VWAP_day5_return_5
2484,2017-09-01,1101,台泥,34.70,34.75,34.50,34.65,-0.57,0.72,4098.0,...,-33.467825,2.169634,12.490180,-53.990059,0.581555,0.287403,0.476697,0.765364,0.681516,0.84
2485,2017-09-04,1101,台泥,34.50,34.60,34.40,34.45,-0.58,0.58,3281.0,...,-27.176140,2.183091,13.013998,-73.467623,0.517180,0.219811,0.348254,0.710035,0.691022,0.47
2486,2017-09-05,1101,台泥,34.40,34.45,34.15,34.25,-0.58,0.87,6737.0,...,-4.080518,2.176526,13.094750,-123.899343,0.392913,0.131305,0.239779,0.589100,0.657048,-0.06
2487,2017-09-06,1101,台泥,34.15,34.20,34.10,34.10,-0.44,0.29,4335.0,...,6.610168,0.787977,11.395635,-79.301344,0.372457,0.062428,0.151104,0.472807,0.595634,-0.74
2488,2017-09-07,1101,台泥,34.10,34.15,33.80,33.80,-0.88,1.03,8073.0,...,35.720149,1.058489,10.888007,-76.598139,0.274473,0.168280,0.159692,0.378879,0.523383,-1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318889,2019-09-02,9958,世紀鋼,69.40,71.60,68.70,71.60,3.32,4.18,1980.0,...,10.399445,30.535589,5.001610,-135.261368,0.433530,0.618687,0.459431,0.251528,0.272107,-2.98
2318890,2019-09-03,9958,世紀鋼,71.70,75.80,71.30,74.10,3.49,6.28,7286.0,...,7.350933,30.535589,35.547803,270.566486,0.669457,0.824588,0.642009,0.472498,0.338904,0.48
2318891,2019-09-04,9958,世紀鋼,75.00,75.40,74.40,74.90,1.08,1.35,4513.0,...,2.819805,30.535589,35.061530,348.691073,0.669457,0.884340,0.763175,0.644186,0.440665,3.30
2318892,2019-09-05,9958,世紀鋼,75.50,75.70,74.00,74.20,-0.93,2.27,2186.0,...,14.334574,30.535589,34.972803,301.521091,0.636482,0.815226,0.789200,0.739897,0.540409,5.25


In [4]:
import json
def read_feature_list(file_path, requirement='whole'):

    assert requirement in ['whole', 'technical', 'buy', 'index']

    d = []
    with open(file_path, 'r') as fp:
        d = json.load(fp)
    
    feature_list = d[requirement]

    return feature_list

feature_list = read_feature_list('D:\\庫存健診開發\\feature_dict.json')

In [11]:
feature_list.append('Y')

In [3]:
df = pd.read_csv('D:\\OneDrive - SinoPac\\training\\test_rolling_new.csv', converters = {'StockNo': str})

In [12]:
test = df[feature_list]
test

,total,capital,index_open,index_high,index_low,index_close,index_return,index_vol,industry_return,foreign_ratio,...,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,industry,ts,StockNo,Y
2484,141911.0,1279.3,10576.27,10619.52,10568.72,10594.82,0.09,6064862.0,-0.18,26.48,...,-53.990059,0.581555,0.287403,0.476697,0.765364,0.681516,水泥工業,2017-09-01,1101,0.0
2485,113227.0,1272.0,10585.44,10610.04,10555.34,10569.87,-0.24,5498749.0,-0.38,26.44,...,-73.467623,0.517180,0.219811,0.348254,0.710035,0.691022,水泥工業,2017-09-04,1101,0.0
2486,230732.0,1264.6,10584.10,10617.84,10564.26,10617.84,0.45,5873609.0,-0.16,26.35,...,-123.899343,0.392913,0.131305,0.239779,0.589100,0.657048,水泥工業,2017-09-05,1101,0.0
2487,148040.0,1259.0,10593.87,10604.92,10519.71,10547.86,-0.66,6210530.0,-0.72,26.36,...,-79.301344,0.372457,0.062428,0.151104,0.472807,0.595634,水泥工業,2017-09-06,1101,0.0
2488,274158.0,1248.0,10564.79,10604.02,10509.91,10538.51,-0.09,6412690.0,-0.46,26.24,...,-76.598139,0.274473,0.168280,0.159692,0.378879,0.523383,水泥工業,2017-09-07,1101,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318889,139540.0,149.9,10613.19,10634.85,10589.42,10634.85,0.16,4333278.0,0.52,1.71,...,-135.261368,0.433530,0.618687,0.459431,0.251528,0.272107,鋼鐵工業,2019-09-02,9958,1.0
2318890,542310.0,155.1,10629.22,10650.15,10557.03,10558.21,-0.72,4043809.0,-0.16,1.68,...,270.566486,0.669457,0.824588,0.642009,0.472498,0.338904,鋼鐵工業,2019-09-03,9958,1.0
2318891,338014.0,156.8,10566.32,10670.43,10566.32,10657.31,0.94,4338123.0,0.12,1.66,...,348.691073,0.669457,0.884340,0.763175,0.644186,0.440665,鋼鐵工業,2019-09-04,9958,1.0
2318892,162831.0,155.3,10718.40,10766.60,10714.27,10756.93,0.93,6013120.0,0.21,1.62,...,301.521091,0.636482,0.815226,0.789200,0.739897,0.540409,鋼鐵工業,2019-09-05,9958,1.0


In [13]:
test['month'] = test['ts'].apply(lambda x: str(x.year) + '-' + str(x.month))
test[['ts', 'month']]

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ts,month
2484,2017-09-01,2017-9
2485,2017-09-04,2017-9
2486,2017-09-05,2017-9
2487,2017-09-06,2017-9
2488,2017-09-07,2017-9
...,...,...
2318889,2019-09-02,2019-9
2318890,2019-09-03,2019-9
2318891,2019-09-04,2019-9
2318892,2019-09-05,2019-9


In [18]:
total_sum = test.groupby(['month','StockNo'])['total'].sum().reset_index().rename(columns={'total': 'total_sum'})
pd.merge(test, total_sum, on=['month', 'StockNo'], how='inner')

,total,capital,index_open,index_high,index_low,index_close,index_return,index_vol,industry_return,foreign_ratio,...,SOk5_1,SOd3,SOk15_5,SOd5,industry,ts,StockNo,Y,month,total_sum
0,141911.0,1279.3,10576.27,10619.52,10568.72,10594.82,0.09,6064862.0,-0.18,26.48,...,0.287403,0.476697,0.765364,0.681516,水泥工業,2017-09-01,1101,0.0,2017-9,4792005.0
1,113227.0,1272.0,10585.44,10610.04,10555.34,10569.87,-0.24,5498749.0,-0.38,26.44,...,0.219811,0.348254,0.710035,0.691022,水泥工業,2017-09-04,1101,0.0,2017-9,4792005.0
2,230732.0,1264.6,10584.10,10617.84,10564.26,10617.84,0.45,5873609.0,-0.16,26.35,...,0.131305,0.239779,0.589100,0.657048,水泥工業,2017-09-05,1101,0.0,2017-9,4792005.0
3,148040.0,1259.0,10593.87,10604.92,10519.71,10547.86,-0.66,6210530.0,-0.72,26.36,...,0.062428,0.151104,0.472807,0.595634,水泥工業,2017-09-06,1101,0.0,2017-9,4792005.0
4,274158.0,1248.0,10564.79,10604.02,10509.91,10538.51,-0.09,6412690.0,-0.46,26.24,...,0.168280,0.159692,0.378879,0.523383,水泥工業,2017-09-07,1101,0.0,2017-9,4792005.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447580,139540.0,149.9,10613.19,10634.85,10589.42,10634.85,0.16,4333278.0,0.52,1.71,...,0.618687,0.459431,0.251528,0.272107,鋼鐵工業,2019-09-02,9958,1.0,2019-9,1491549.0
447581,542310.0,155.1,10629.22,10650.15,10557.03,10558.21,-0.72,4043809.0,-0.16,1.68,...,0.824588,0.642009,0.472498,0.338904,鋼鐵工業,2019-09-03,9958,1.0,2019-9,1491549.0
447582,338014.0,156.8,10566.32,10670.43,10566.32,10657.31,0.94,4338123.0,0.12,1.66,...,0.884340,0.763175,0.644186,0.440665,鋼鐵工業,2019-09-04,9958,1.0,2019-9,1491549.0
447583,162831.0,155.3,10718.40,10766.60,10714.27,10756.93,0.93,6013120.0,0.21,1.62,...,0.815226,0.789200,0.739897,0.540409,鋼鐵工業,2019-09-05,9958,1.0,2019-9,1491549.0


In [19]:
total_sum = test.groupby(['month','StockNo'])['total'].sum().reset_index().rename(columns={'total': 'total_sum'})
df = pd.merge(test, total_sum, on=['month', 'StockNo'], how='inner')
df_list = [group[1] for group in df.groupby(df['month'])]
output_list = []
for d in df_list:
    d['PctRank'] = d['total_sum'].rank(pct=True)
    output_list.append(d)
    
df = pd.concat(output_list, axis=0)
df

,total,capital,index_open,index_high,index_low,index_close,index_return,index_vol,industry_return,foreign_ratio,...,SOd3,SOk15_5,SOd5,industry,ts,StockNo,Y,month,total_sum,PctRank
22,128253.0,1242.4,10425.10,10472.81,10414.53,10465.16,0.78,4720526.0,-0.04,25.27,...,0.311435,0.276193,0.329356,水泥工業,2017-10-02,1101,1.0,2017-10,2599779.0,0.817531
23,81214.0,1244.3,10462.70,10495.34,10445.86,10469.35,0.04,4790980.0,-0.28,25.26,...,0.213681,0.208967,0.289226,水泥工業,2017-10-03,1101,1.0,2017-10,2599779.0,0.817531
24,128765.0,1248.0,10473.28,10530.93,10473.28,10518.27,0.47,4672634.0,0.17,25.26,...,0.296272,0.166213,0.248222,水泥工業,2017-10-05,1101,1.0,2017-10,2599779.0,0.817531
25,98897.0,1246.1,10538.21,10557.67,10511.50,10532.81,0.14,4669026.0,0.17,25.26,...,0.406242,0.096022,0.197488,水泥工業,2017-10-06,1101,1.0,2017-10,2599779.0,0.817531
26,323234.0,1255.3,10576.12,10668.49,10576.12,10641.19,1.03,5950659.0,0.75,25.22,...,0.533930,0.357384,0.250787,水泥工業,2017-10-11,1101,1.0,2017-10,2599779.0,0.817531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447580,139540.0,149.9,10613.19,10634.85,10589.42,10634.85,0.16,4333278.0,0.52,1.71,...,0.459431,0.251528,0.272107,鋼鐵工業,2019-09-02,9958,1.0,2019-9,1491549.0,0.912707
447581,542310.0,155.1,10629.22,10650.15,10557.03,10558.21,-0.72,4043809.0,-0.16,1.68,...,0.642009,0.472498,0.338904,鋼鐵工業,2019-09-03,9958,1.0,2019-9,1491549.0,0.912707
447582,338014.0,156.8,10566.32,10670.43,10566.32,10657.31,0.94,4338123.0,0.12,1.66,...,0.763175,0.644186,0.440665,鋼鐵工業,2019-09-04,9958,1.0,2019-9,1491549.0,0.912707
447583,162831.0,155.3,10718.40,10766.60,10714.27,10756.93,0.93,6013120.0,0.21,1.62,...,0.789200,0.739897,0.540409,鋼鐵工業,2019-09-05,9958,1.0,2019-9,1491549.0,0.912707


In [21]:
df['PctRank'].min()

5.391707553782283e-05

In [22]:
df.to_csv('D:\\OneDrive - SinoPac\\training\\test_rolling_without_MACD_whole.csv', index=False)

In [3]:
train_df = pd.read_csv('D:\\庫存健診開發\\data\\Clustering\\train_cluster.csv', index_col=0)
test_df = pd.read_csv('D:\\庫存健診開發\\data\\Clustering\\test_cluster.csv', index_col=0)

In [4]:
file_path = 'D:\\庫存健診開發\\data\\Training\\Raw\\Whole_day5_'
func.total_split(train_df, test_df, data, file_path)

D:\庫存健診開發\code\TrainingPrep\Training_function.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['cluster'] = i
D:\庫存健診開發\code\TrainingPrep\Training_function.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['cluster'] = i


,ts,StockNo,StockName,open,high,low,close,return,volatility,vol,...,investment_pvt,dealer_pvt,pvt,RSI_15,SOk5_1,SOd3,SOk15_5,SOd5,VWAP_day5_return_5,cluster
224499,2007-08-21,1452,宏益,10.5,10.50,10.10,10.15,-1.93,3.86,715.0,...,0.000000,0.198977,-432.370845,0.352887,0.306385,0.263477,0.315794,0.472183,-13.17,0
224500,2007-08-22,1452,宏益,10.2,10.25,10.05,10.25,0.99,1.97,307.0,...,0.000000,0.232083,-331.519276,0.346320,0.419928,0.341702,0.214204,0.386190,-15.15,0
224501,2007-08-23,1452,宏益,10.4,10.70,10.35,10.70,4.39,3.41,535.0,...,0.000000,0.273522,-194.180321,0.451337,0.891234,0.616468,0.198582,0.323654,-12.60,0
224502,2007-08-24,1452,宏益,10.8,10.80,10.40,10.50,-1.87,3.74,727.0,...,0.000000,-0.111481,87.923127,0.472845,0.686039,0.651253,0.277556,0.308288,-8.42,0
224503,2007-08-27,1452,宏益,10.8,10.80,10.40,10.40,-0.95,3.81,317.0,...,0.000000,-0.148563,100.341491,0.472845,0.707256,0.679254,0.304387,0.306987,-3.23,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2318889,2019-09-02,9958,世紀鋼,69.4,71.60,68.70,71.60,3.32,4.18,1980.0,...,30.535589,5.001610,-135.261368,0.433530,0.618687,0.459431,0.251528,0.272107,-2.98,4
2318890,2019-09-03,9958,世紀鋼,71.7,75.80,71.30,74.10,3.49,6.28,7286.0,...,30.535589,35.547803,270.566486,0.669457,0.824588,0.642009,0.472498,0.338904,0.48,4
2318891,2019-09-04,9958,世紀鋼,75.0,75.40,74.40,74.90,1.08,1.35,4513.0,...,30.535589,35.061530,348.691073,0.669457,0.884340,0.763175,0.644186,0.440665,3.30,4
2318892,2019-09-05,9958,世紀鋼,75.5,75.70,74.00,74.20,-0.93,2.27,2186.0,...,30.535589,34.972803,301.521091,0.636482,0.815226,0.789200,0.739897,0.540409,5.25,4
